In [1]:
import platform
print(platform.machine())


AMD64


In [2]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 2060
Using device: cuda


In [3]:
print("CUDA Available:", torch.cuda.is_available())
print("Current GPU:", torch.cuda.current_device())
print("GPU Memory Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("GPU Memory Cached:", torch.cuda.memory_reserved() / 1024**2, "MB")

CUDA Available: True
Current GPU: 0
GPU Memory Allocated: 0.0 MB
GPU Memory Cached: 0.0 MB


In [4]:
import torch

# Create a tensor and move it to GPU
x = torch.rand(3, 3).to("cuda")

print("Tensor on GPU:", x)
print("GPU Memory Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
print("GPU Memory Cached:", torch.cuda.memory_reserved() / 1024**2, "MB")

Tensor on GPU: tensor([[0.1331, 0.8529, 0.1293],
        [0.3817, 0.6938, 0.1101],
        [0.6866, 0.7192, 0.0525]], device='cuda:0')
GPU Memory Allocated: 0.00048828125 MB
GPU Memory Cached: 2.0 MB
